In [3]:
import pandas as pd
import numpy as np
from surprise import Dataset
from surprise import Reader
from surprise.model_selection import train_test_split
from surprise import SVD
import gradio as gr

In [19]:
# Este código es el resumen de lo explicado en el archivo "Proceso ML" para predecir.

reader = Reader()
df=pd.read_csv("datasets/ratings_ml.csv")
df_ratings = df.sort_values(by="userId", ascending=True)
titulos = pd.read_csv("datasets/titulos.csv")
num_filas= 100000
data = Dataset.load_from_df(df_ratings[["userId","IdMovie","rating"]][:num_filas],reader)
trainset, testset = train_test_split(data, test_size=.25)
model = SVD()
model.fit(trainset)
prediction=model.test(testset)

In [20]:
def recomendacion(usuario, movie):
    '''
    Esta función aplica el modelo predictivo a un usario específco con respecto a una película específica.
    Args:
        usario: variable tipo entero que refiere al id del usuario que se quiere buscar.
        movie: variabele tipo entero que refiere al id de la película que se quiere predecir.
    Returns:
        Un string que valida si la película se recomienda dentro del topten de sugerencias para el usuario.
    '''
    #Aseguramos el tipo de dato que requerimos.
    usuario = int(usuario)
    movie = int(movie)
    #filtramos de las bases si el usario y la película se encuentran en las bases que tenemos.
    validacion_usuario = df_ratings[df_ratings["userId"] == usuario]
    validacion_movie = titulos[titulos["IdMovie"]==movie]
        
    if validacion_usuario.shape[0] > 0: #validamos si se encuentra al menos un registro del id de usuario
        validacion_usuario = True
    else: return f"El usuario con id {usuario} no se encuentra disponible"

    if validacion_movie.shape[0] > 0: #validamos si se encuentra al menos un registro del id de la pelicula
        validacion_movie = True
    else: return f"La pelicula con id {movie} no existe en el catálogo"

    #Filtramos la recomendación contemplando que el usuario haya puntuado con al menos 4 puntos
    df_usuario = df_ratings[(df_ratings['userId']==usuario)&(df_ratings['rating']>=4)]    
    df_usuario = df_usuario.reset_index(drop=True)
    df_usuario['Titulo'] = titulos['title'].loc[df_usuario.IdMovie].values

    # Copiamos los titulos en otra variable para que no se afecte el dataframe original de titulos
    recomendaciones = titulos.copy()

    #Extraemos las peliculas que ya ha visto.
    usuario_vistas = df_ratings[df_ratings['userId']== usuario]

    #Eliminamos las que ya vio de las recomendaciones (los titulos)
    recomendaciones.drop(usuario_vistas.IdMovie, inplace=True)  

    #Realizamos las recomendaciones
    recomendaciones['Puntaje_Estimado']=recomendaciones['IdMovie'].apply(lambda x: model.predict(usuario,x).est)

    #Ordenamos las puntuaciones de mayor a menor
    recomendaciones = recomendaciones.sort_values('Puntaje_Estimado', ascending = False)

    #Nos quedamos con el topten
    df_recom = recomendaciones.head(10)

    #Contamos las veces que el Id de la película solicitada esté en el topten
    recom = df_recom[(df_recom["IdMovie"]==movie)]

    #Validamos si encontramos el registro y devolvemos el string que corresponda.
    if recom.shape[0] > 0: 
        return f"SÍ se recomienda la pelicula con id {movie} para el usuario {usuario}"
    else:   
        return f"NO se recomienda la pelicula con id {movie} para el usuario {usuario}"

# Hacemos el deploy local
demo = gr.Interface(fn=recomendacion, inputs=["text", "text"], outputs="text")
demo.launch()


Running on local URL:  http://127.0.0.1:7863

To create a public link, set `share=True` in `launch()`.


In [21]:
user_test = 583

In [22]:
def recomendacion_valid(usuario):
    validacion_usuario = df_ratings[df_ratings["userId"] == usuario]
        
    if validacion_usuario.shape[0] > 0:
        validacion_usuario = True
    else: return f"El usuario con id {usuario} no se encuentra disponible"

    df_usuario = df_ratings[(df_ratings['userId']==usuario)&(df_ratings['rating']>=4)]    
    df_usuario = df_usuario.reset_index(drop=True)
    df_usuario['Titulo'] = titulos['title'].loc[df_usuario.IdMovie].values

    # Copiamos los titulos en otra variable para que no se afecte el dataframe original de titulos
    recomendaciones = titulos.copy()

    #Extraemos las peliculas que ya ha visto.
    usuario_vistas = df_ratings[df_ratings['userId']== usuario]
 
    #Eliminamos las que ya vio de las recomendaciones (los titulos)
    recomendaciones.drop(usuario_vistas.IdMovie, inplace=True)  

    #Realizamos las recomendaciones
    recomendaciones['Puntaje_Estimado']=recomendaciones['IdMovie'].apply(lambda x: model.predict(usuario,x).est)

    recomendaciones = recomendaciones.sort_values('Puntaje_Estimado', ascending = False)

    df_recom = recomendaciones.head(10)

    return df_recom

In [14]:
recomendacion_valid(user_test)

,IdMovie,title,Puntaje_Estimado
11296,9741,the simpsons,4.768065
9760,9741,the simpsons,4.768065
4816,4809,100 days to live,4.760185
17725,17182,the spy,4.714671
8231,8220,the wheel of time,4.688852
15077,14702,the mystic river,4.681753
11493,11428,women's college soccer,4.635474
9151,9137,man from del rio,4.621291
20396,19724,babel,4.620835
9440,9424,nicholas nickleby,4.617103


In [18]:
prueba = titulos.head()
type(prueba)

pandas.core.frame.DataFrame